In [2]:
# 计算 U 统计量的样本方差
import itertools
import numpy as np
# 示例数据
data = [4.0, 7.2, 5.5, 3.1, 6.8, 9.1]
# 1. U-统计量方法
# 定义核函数 h(x,y) = 0.5*(x - y)^2
def h_var(x, y):
    return 0.5 * (x - y)**2
n = len(data)
m = 2  # 核函数阶数
if n < m:
    raise ValueError("Sample size must be at least m.")
# 使用 itertools.combinations 生成所有不重复的二元组索引
combinations = list(itertools.combinations(range(n), m))
h_values = [h_var(data[i], data[j]) for i, j in combinations]
# 计算U统计量
U_var = sum(h_values) / len(combinations)
print(f"U-statistic for variance: {U_var:.4f}")
# 2. 对比直接计算无偏样本方差
# ddof=1 表示分母为 n-1
direct_var = np.var(data, ddof=1)
print(f"Direct sample variance (S^2): {direct_var:.4f}")
assert np.isclose(U_var, direct_var)

U-statistic for variance: 4.8670
Direct sample variance (S^2): 4.8670


In [7]:
# 使用自助法估计P(X < Y)的95%置信区间
import numpy as np
from scipy.stats import norm
rng = np.random.default_rng(seed=1) 
n = 60  # 样本X的大小
m = 100  # 样本Y的大小
x_sample = rng.normal(loc=0, scale=1, size=n)
y_sample = rng.normal(loc=1, scale=1, size=m)
# 1. 定义一个函数来计算我们的目标统计量 P(X < Y) 的估计值
def calculate_p_xy(x_data, y_data):
    n_local = len(x_data)
    m_local = len(y_data)
    if n_local == 0 or m_local == 0:
        return 0.0 # 避免除以零
    comparison_matrix = x_data[:, np.newaxis] < y_data
    u_statistic = np.sum(comparison_matrix)
    return u_statistic / (n_local * m_local)
# 2. Bootstrap 循环
B = 5000  # Bootstrap 重复次数
bootstrap_results = np.empty(B) # 预先分配数组以提高效率
for i in range(B):
    x_star = rng.choice(x_sample, size=n, replace=True)
    y_star = rng.choice(y_sample, size=m, replace=True)
    bootstrap_results[i] = calculate_p_xy(x_star, y_star)
# 3. 构建置信区间 (百分位法)
alpha = 0.05  # 显著性水平，对于95%置信区间，alpha=0.05
lower_bound = np.percentile(bootstrap_results, 100 * (alpha / 2))
upper_bound = np.percentile(bootstrap_results, 100 * (1 - alpha / 2))
# 4. 结果展示
original_estimate = calculate_p_xy(x_sample, y_sample)
print(f"原始样本的 P(X < Y) 估计值: {original_estimate:.4f}")
# >>> 原始样本的 P(X < Y) 估计值: 0.7728
print(f"95% Bootstrap 置信区间 (百分位法): ({lower_bound:.4f}, {upper_bound:.4f})")
#>>> 95% Bootstrap 置信区间 (百分位法): (0.6987, 0.8427)
# 计算理论值
mean_y = np.mean(y_sample)
std_y = np.std(y_sample)
mean_x = np.mean(x_sample)
std_x = np.std(x_sample)
true_mean_z = mean_y - mean_x
true_std_z = np.sqrt(std_y**2 + std_x**2)
# P(Z > 0) = 1 - P(Z <= 0) = 1 - CDF(0)
true_p_xy = 1 - norm.cdf(0, loc=true_mean_z, scale=true_std_z)
print(f"P(X < Y) 的理论值: {true_p_xy:.4f}")

原始样本的 P(X < Y) 估计值: 0.7728
95% Bootstrap 置信区间 (百分位法): (0.6987, 0.8427)
P(X < Y) 的理论值: 0.7691


In [4]:
# 使用自助法估计 Var(X) 的95%置信区间
import numpy as np
from scipy.stats import norm
rng = np.random.default_rng(seed=1) 
n = 40  # 样本X的大小
mu = 0
sigma_sq = 4
x_sample = rng.normal(loc=mu, scale=np.sqrt(sigma_sq), size=n)
# 1. 定义一个函数来计算我们的目标统计量 Var(X) 的估计值
def calculate_variance(data):
    if len(data) < 2:
        return 0.0  # 避免除以零或负自由度
    return np.var(data, ddof=1)
# 2. Bootstrap 循环
B = 5000  # Bootstrap 重复次数
bootstrap_results = np.empty(B) # 预先分配数组以提高效率
for i in range(B):
    x_star = rng.choice(x_sample, size=n, replace=True)
    bootstrap_results[i] = calculate_variance(x_star)
# 3. 构建置信区间 (百分位法)
alpha = 0.05  # 显著性水平，对于95%置信区间，alpha=0.05
lower_bound = np.percentile(bootstrap_results, 100 * (alpha / 2))
upper_bound = np.percentile(bootstrap_results, 100 * (1 - alpha / 2))
# 4. 结果展示
original_estimate = calculate_variance(x_sample)
print(f"原始样本的 Var(X) 估计值: {original_estimate:.4f}")
# >>> 原始样本的 Var(X) 估计值: 3.5477
print(f"95% Bootstrap 置信区间 (百分位法): ({lower_bound:.4f}, {upper_bound:.4f})")
# >>> 95% Bootstrap 置信区间 (百分位法): (1.7390, 5.5055)

原始样本的 Var(X) 估计值: 3.5477
95% Bootstrap 置信区间 (百分位法): (1.7390, 5.5055)
